In [6]:
# Import libraries
import ccxt
import pandas as pd
from ta.momentum import RSIIndicator, StochasticOscillator
from ta.trend import MACD, SMAIndicator, EMAIndicator
from ta.volatility import BollingerBands, AverageTrueRange

# ตั้งค่าการเชื่อมต่อ Binance
exchange = ccxt.binance()
symbol = 'BTC/USDT'  # ใช้ USDT แทน USD เพราะ Binance ใช้ stablecoin
timeframe = '1h'  # กรอบเวลา 1 ชั่วโมง

# ดึงข้อมูล OHLCV ย้อนหลัง 2160 ชั่วโมง
data = exchange.fetch_ohlcv(symbol, timeframe, limit=2160)
df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# คำนวณ RSI (Relative Strength Index)
rsi = RSIIndicator(close=df['close'], window=14)
df['rsi'] = rsi.rsi()

# คำนวณ MACD (Moving Average Convergence Divergence)
macd = MACD(close=df['close'])
df['macd'] = macd.macd()
df['macd_signal'] = macd.macd_signal()
df['macd_diff'] = macd.macd_diff()

# คำนวณ Bollinger Bands (BB)
bb = BollingerBands(close=df['close'], window=20, window_dev=2)
df['bb_upper'] = bb.bollinger_hband()
df['bb_lower'] = bb.bollinger_lband()
df['bb_middle'] = bb.bollinger_mavg()

# คำนวณ SMA (Simple Moving Average) และ EMA (Exponential Moving Average)
df['sma_50'] = SMAIndicator(close=df['close'], window=50).sma_indicator()
df['ema_20'] = EMAIndicator(close=df['close'], window=20).ema_indicator()

# คำนวณ Stochastic Oscillator (STOCH)
stoch = StochasticOscillator(high=df['high'], low=df['low'], close=df['close'], window=14, smooth_window=3)
df['stoch_k'] = stoch.stoch()
df['stoch_d'] = stoch.stoch_signal()

# คำนวณ ATR (Average True Range)
atr = AverageTrueRange(high=df['high'], low=df['low'], close=df['close'], window=14)
df['atr'] = atr.average_true_range()

# Drop missing value from the dataset
df.dropna(inplace=True)

#สร้าง Target Variable ถ้าราคาปิดของอีกวันสูงขึ้นจะเป็นการ บ่งบอกว่า Good Target
df['target'] = (df['close'].shift(-1) > df['close']).astype(int)

# การเปลี่ยนแปลงของราคา
df['price_change'] = df['close'] - df['open']

In [7]:
# Time Series Train/Test Split (ใช้ 80% สำหรับ Train และ 20% สำหรับ Test)
split_ratio = 0.8
split_index = int(len(df) * split_ratio)

train_df = df.iloc[:split_index]  # 80% แรกเป็น Train
test_df = df.iloc[split_index:]   # 20% ที่เหลือเป็น Test

# แยก Features และ Target
features = ['close', 'rsi', 'macd', 'macd_signal', 'macd_diff', 
            'bb_upper', 'bb_middle', 'bb_lower', 'sma_50', 'ema_20', 
            'stoch_k', 'stoch_d', 'atr', 'price_change']

X_train = train_df[features]
y_train = train_df['target']

X_test = test_df[features]
y_test = test_df['target']

In [8]:
from sklearn.preprocessing import StandardScaler

# Standardization (Z-score Normalization)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Fit และ Transform สำหรับ Train
X_test_scaled = scaler.transform(X_test)  # Transform สำหรับ Test

In [18]:
import xgboost as xgb
from sklearn.feature_selection import RFE
from sklearn.metrics import precision_score, classification_report
import numpy as np

# 1. คำนวณ scale_pos_weight เพื่อจัดการ Class Imbalance
class_0 = sum(y_train == 0)
class_1 = sum(y_train == 1)
scale_pos_weight = class_0 / class_1
print(f"scale_pos_weight: {scale_pos_weight}")

# 2. สร้างโมเดล XGBoost พร้อม scale_pos_weight
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.001,
    scale_pos_weight=scale_pos_weight,
    random_state=42
)

# 3. ใช้ RFE เพื่อเลือก Features ที่สำคัญ
selector = RFE(estimator=xgb_model, n_features_to_select=7, step=1)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# 4. Train โมเดล
xgb_model.fit(X_train_selected, y_train)

# 5. พยากรณ์ Test Set ด้วย Threshold ปรับค่า
y_prob = xgb_model.predict_proba(X_test_selected)[:, 1]
threshold = 0.5
y_pred_adjusted = (y_prob > threshold).astype(int)

# 6. ประเมินผลลัพธ์
print("\nClassification Report (Adjusted Threshold):")
print(classification_report(y_test, y_pred_adjusted))

# 7. นำค่าข้อมูลชั่วโมงสุดท้ายมาใช้ทำนายจริง
latest_data = X_test_selected[-1].reshape(1, -1)  # นำค่าล่าสุด (ชั่วโมงสุดท้าย)
latest_prob = xgb_model.predict_proba(latest_data)[:, 1]  # คำนวณ Probabilities
latest_pred = (latest_prob > threshold).astype(int)  # ตัดสินใจตาม Threshold

print("\n🔮 Prediction for Latest Hour 🔮")
print(f"Predicted Probability: {latest_prob[0]:.4f}")
print(f"Predicted Class (Threshold {threshold}): {latest_pred[0]}")


scale_pos_weight: 0.9537275064267352

Classification Report (Adjusted Threshold):
              precision    recall  f1-score   support

           0       0.43      0.44      0.43       103
           1       0.33      0.33      0.33        88

    accuracy                           0.39       191
   macro avg       0.38      0.38      0.38       191
weighted avg       0.39      0.39      0.39       191


🔮 Prediction for Latest Hour 🔮
Predicted Probability: 0.4825
Predicted Class (Threshold 0.5): 0
